In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
from collections import deque, Counter

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

def angle3pt(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ab = a - b
    cb = c - b
    cosine_angle = np.dot(ab, cb) / ((np.linalg.norm(ab) * np.linalg.norm(cb)) + 1e-9)
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

def classify_swimming_style(landmarks):
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

    left_arm_angle = angle3pt(left_wrist, left_elbow, left_shoulder)
    right_arm_angle = angle3pt(right_wrist, right_elbow, right_shoulder)

    if 160 < left_arm_angle < 180 and 160 < right_arm_angle < 180:
        return "Backstroke"
    elif left_arm_angle < 90 and right_arm_angle < 90:
        return "Butterfly"
    elif 100 < left_arm_angle < 140 and 100 < right_arm_angle < 140:
        return "Freestyle"
    elif 140 <= left_arm_angle < 160 and 140 <= right_arm_angle < 160:
        return "Breaststroke"
    else:
        return ""

def process_video(video_path, output_path):
    if not os.path.exists(video_path):
        print("Invalid input video path!")
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open the video.")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    recent_styles = deque(maxlen=10)
    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)

        if result.pose_landmarks:
            mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            style = classify_swimming_style(result.pose_landmarks.landmark)
            recent_styles.append(style)

            common_style = Counter(recent_styles).most_common(1)[0][0] if recent_styles else style

            cv2.putText(frame, f"Style: {common_style}", (30, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        out.write(frame)

        frame_index += 1
        if frame_index % 10 == 0 or frame_index == frame_count:
            print(f"Processing frame {frame_index}/{frame_count}", end="\r")

    cap.release()
    out.release()
    print("\nProcessing complete. Output saved to:", output_path)

def main():
    video_path = ".mp4"  # Replace with your input path
    output_path = "output_videos/FL_pose_output.mp4"   # Replace with desired output path

    process_video(video_path, output_path)

if __name__ == "__main__":
    main()
